## Inona mo ny principe ny ML
- Préparation des données : **nettoyage, encoding, scaling**: OneHot, SimpleImputer, LabelEncoding, StandardScaler
- Split train/test : 80/20 ou 70/30
- Entraînement : **model.fit(X_train, y_train)**
- Prédiction : **y_pred = model.predict(X_test)**
- Évaluation avec métriques

## Quels sont les differents metriques

## REGRESSION LINEAIRE

In [ ]:
# 1. IMPORT Regression
# 📊 1. RÉGRESSION (Prédire un NOMBRE)
# Quand ?
# Prédire valeur continue : prix, température, quantité, âge...
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# 2. PRÉPARATION DES DONNÉES
# - Gérer valeurs manquantes
df.fillna(df.mean(), inplace=True)

# - Encoder variables catégorielles (si nécessaire)
# Pour régression : souvent LabelEncoder ou OneHotEncoder
le = LabelEncoder()
df['ville'] = le.fit_transform(df['ville'])

# OU OneHotEncoder pour créer colonnes binaires
df_encoded = pd.get_dummies(df, columns=['ville'], drop_first=True)

# - Séparer X et y
X = df.drop('prix', axis=1)  # Features
y = df['prix']                # Target (variable continue)

# 3. SPLIT
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 4. SCALING (Important pour régression !)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Attention : transform seulement !

# 5. ENTRAÎNEMENT
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# 6. PRÉDICTION
y_pred = model.predict(X_test_scaled)

# 7. ÉVALUATION
print(f"R² : {r2_score(y_test, y_pred):.3f}")  # 0 à 1 (proche de 1 = bon)
print(f"RMSE : {mean_squared_error(y_test, y_pred, squared=False):.2f}")
print(f"MAE : {mean_absolute_error(y_test, y_pred):.2f}")

## 🎯 3. RÉGRESSION LOGISTIQUE (Classification Binaire)
Quand ?
Classification binaire avec probabilités : churn oui/non, clic oui/non...

In [ ]:
# 1. IMPORT
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# 2-3. PRÉPARATION et SPLIT (identique à classification)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. SCALING (OBLIGATOIRE pour Logistic Regression !)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. ENTRAÎNEMENT
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_scaled, y_train)

# 6. PRÉDICTION
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]  # Probabilités classe 1

# 7. ÉVALUATION
print(f"Accuracy : {accuracy_score(y_test, y_pred):.3f}")
print(f"AUC-ROC : {roc_auc_score(y_test, y_proba):.3f}")  # Spécifique logistique
print(classification_report(y_test, y_pred))

In [ ]:
## 🌳 ARBRES DE DÉCISION & FORÊTS ALÉATOIRES
### 1. ARBRE DE DÉCISION (Decision Tree)
Concept simple
C'est comme un organigramme de questions qui divise les données étape par étape.
Exemple concret : Prédire si quelqu'un va acheter un produit
                    Âge < 30 ans ?
                   /              \
                OUI                NON
                 /                  \
        Salaire > 2000€ ?      Marié ?
           /        \           /     \
        OUI        NON        OUI    NON
         /          \          /       \
    ACHÈTE    ACHÈTE PAS   ACHÈTE  ACHÈTE PAS


### Comment ça marche ?

- L'algorithme cherche la meilleure question à poser (feature qui sépare le mieux les classes)
- Il divise les données selon la réponse
- Il répète le processus sur chaque sous-groupe
- Il s'arrête quand les groupes sont "purs" ou selon critères définis

#### Avantages ✅

- Facile à visualiser et interpréter
- Pas besoin de scaling **(StandardScaler)**
- Gère variables **catégorielles et numériques**
- Gère les non-linéarités naturellement

#### Inconvénients ❌

- SURAPPRENTISSAGE (overfitting) très facile !
- Instable : petits changements données → arbre très différent
- Performances moyennes seul

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(
    max_depth=5,           # Profondeur max (limite overfitting) ⭐
    min_samples_split=10,  # Min échantillons pour diviser nœud
    min_samples_leaf=5,    # Min échantillons par feuille
    max_features='sqrt',   # Nombre features à considérer
    random_state=42
)
```

**Paramètres clés à retenir** :
- **max_depth** : Plus important ! Limite la profondeur (3-10 généralement)
- **min_samples_split** : Évite divisions sur petits groupes
- **min_samples_leaf** : Évite feuilles trop spécifiques

---



## **2. FORÊT ALÉATOIRE (Random Forest)**

### **Concept simple**
C'est un **ensemble de nombreux arbres** qui votent ensemble !

**Analogie** : Au lieu de demander à 1 expert, tu demandes à 100 experts et tu prends la réponse majoritaire.
```
Arbre 1: ACHÈTE       │
Arbre 2: ACHÈTE       │
Arbre 3: ACHÈTE PAS   │  VOTE MAJORITAIRE
Arbre 4: ACHÈTE       │  → ACHÈTE (75%)
...                   │
Arbre 100: ACHÈTE     │

#### Comment ça marche ?

- Crée plusieurs arbres (100, 500, 1000...)
- Chaque arbre est entraîné sur un échantillon aléatoire des données **(bootstrap)**
- Chaque arbre utilise un sous-ensemble aléatoire de features à chaque division
- Prédiction finale = vote majoritaire (classification) ou moyenne (régression)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=100,      # Nombre d'arbres (100-500) ⭐⭐⭐
    max_depth=10,          # Profondeur max de chaque arbre ⭐
    min_samples_split=5,   # Min échantillons pour diviser
    min_samples_leaf=2,    # Min échantillons par feuille
    max_features='sqrt',   # Features aléatoires par division ⭐
    random_state=42,
    n_jobs=-1              # Utilise tous les CPU (parallélisation)
)

In [ ]:
Paramètres clés à retenir :

n_estimators : Plus = mieux (mais plus lent). Start avec 100.
max_depth : Contrôle overfitting (None par défaut, mais limiter à 10-20 souvent mieux)
max_features : 'sqrt' pour classification, 'log2' alternative

In [ ]:
# ========== IMPORTS ==========
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

# ========== DONNÉES EXEMPLE ==========
# Créons un dataset simple
from sklearn.datasets import load_iris
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ========== 1. ARBRE DE DÉCISION ==========
print("=" * 50)
print("ARBRE DE DÉCISION")
print("=" * 50)

# Modèle
dt_model = DecisionTreeClassifier(
    max_depth=3,              # Limite profondeur (évite overfitting)
    min_samples_split=5,
    random_state=42
)

# Entraînement
dt_model.fit(X_train, y_train)

# Prédiction
y_pred_dt = dt_model.predict(X_test)

# Évaluation
print(f"Accuracy : {accuracy_score(y_test, y_pred_dt):.3f}")
print(f"F1-Score : {f1_score(y_test, y_pred_dt, average='weighted'):.3f}")

# Cross-validation pour vérifier overfitting
cv_scores_dt = cross_val_score(dt_model, X_train, y_train, cv=5)
print(f"Cross-Val Accuracy : {cv_scores_dt.mean():.3f} (+/- {cv_scores_dt.std():.3f})")

# Visualiser l'arbre (optionnel)
plt.figure(figsize=(15, 10))
plot_tree(dt_model, 
          feature_names=iris.feature_names,
          class_names=iris.target_names,
          filled=True,
          rounded=True,
          fontsize=10)
plt.title("Arbre de Décision")
plt.tight_layout()
plt.savefig('decision_tree.png')
print("Arbre sauvegardé dans 'decision_tree.png'")

# ========== 2. FORÊT ALÉATOIRE ==========
print("\n" + "=" * 50)
print("FORÊT ALÉATOIRE")
print("=" * 50)

# Modèle
rf_model = RandomForestClassifier(
    n_estimators=100,         # 100 arbres
    max_depth=5,              # Profondeur max
    min_samples_split=5,
    max_features='sqrt',      # Features aléatoires
    random_state=42,
    n_jobs=-1                 # Parallélisation
)

# Entraînement
rf_model.fit(X_train, y_train)

# Prédiction
y_pred_rf = rf_model.predict(X_test)

# Évaluation
print(f"Accuracy : {accuracy_score(y_test, y_pred_rf):.3f}")
print(f"F1-Score : {f1_score(y_test, y_pred_rf, average='weighted'):.3f}")

# Cross-validation
cv_scores_rf = cross_val_score(rf_model, X_train, y_train, cv=5)
print(f"Cross-Val Accuracy : {cv_scores_rf.mean():.3f} (+/- {cv_scores_rf.std():.3f})")

# Rapport détaillé
print("\nClassification Report (Random Forest):")
print(classification_report(y_test, y_pred_rf, target_names=iris.target_names))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

# ========== 3. IMPORTANCE DES FEATURES ==========
print("\n" + "=" * 50)
print("IMPORTANCE DES FEATURES (Random Forest)")
print("=" * 50)

feature_importance = pd.DataFrame({
    'Feature': iris.feature_names,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(feature_importance)

# Visualiser importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Importance'])
plt.xlabel('Importance')
plt.title('Importance des Features')
plt.tight_layout()
plt.savefig('feature_importance.png')
print("\nGraphique sauvegardé dans 'feature_importance.png'")

# ========== 4. COMPARAISON ==========
print("\n" + "=" * 50)
print("COMPARAISON")
print("=" * 50)

comparison = pd.DataFrame({
    'Modèle': ['Decision Tree', 'Random Forest'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_dt),
        accuracy_score(y_test, y_pred_rf)
    ],
    'F1-Score': [
        f1_score(y_test, y_pred_dt, average='weighted'),
        f1_score(y_test, y_pred_rf, average='weighted')
    ],
    'CV Mean': [cv_scores_dt.mean(), cv_scores_rf.mean()],
    'CV Std': [cv_scores_dt.std(), cv_scores_rf.std()]
})

print(comparison)

5. ÉLÉMENTS À CONNAÎTRE ABSOLUMENT
Pour Arbre de Décision 🌳

max_depth : TOUJOURS limiter (3-10) sinon overfitting garanti
Pas besoin de StandardScaler
Facilement visualisable avec plot_tree()
Bon pour comprendre les données, moins pour production

Pour Random Forest 🌲🌲🌲

n_estimators : Plus = mieux (100 minimum, 500 optimal souvent)
max_depth : Limiter aussi (10-20) même si moins critique
feature_importances_ : Indique quelles features sont importantes
Out-of-bag (OOB) score : Validation automatique sans split
Meilleur choix pour la plupart des problèmes réels

Critères de division (théorie)

Gini : Par défaut, mesure impureté des nœuds
Entropy : Alternative (information gain)
Ne pas trop s'en soucier, Gini fonctionne bien

# Tips

In [ ]:
# 1. Toujours faire cross-validation pour détecter overfitting
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_train, y_train, cv=5)
print(f"CV: {scores.mean():.3f} (+/- {scores.std():.3f})")

# 2. GridSearchCV pour trouver meilleurs paramètres
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10]
}
grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid.fit(X_train, y_train)
print(f"Meilleurs params: {grid.best_params_}")

# 3. Feature importance pour feature selection
importances = rf_model.feature_importances_
# Garder seulement features importantes (> seuil)
important_features = X.columns[importances > 0.05]